In [101]:
from LambertSolarSystem import LambertSolarSystem
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from PyKEP import AU, DAY2SEC, DAY2YEAR, MU_SUN, epoch
from PyKEP.orbit_plots import plot_planet, plot_lambert
import PyKEP as pkp
import numpy as np

first_boost = 9594



vel_at_closest = 0
SEC2YEAR = 1./DAY2YEAR*DAY2SEC

system = LambertSolarSystem(87464)
t = 14.32
A = 0; B = 1
planets = np.array((A,B))
a, e, i, W, w, M  = system.getKeplerianElements()[:,planets]
M = system.getM(t)[planets]
posFunc = system.getPositionsFunction()

kep_elements = np.array((a*AU,e,i,W,w,M))

mu_star = system.starMass * MU_SUN
mu_planet = system.mass[planets]  * MU_SUN
radius = system.radius[planets] * 1000
safe_radius = radius *1.2
planetA = pkp.planet.keplerian(epoch(0), kep_elements[:,0], mu_star, mu_planet[0], radius[0], safe_radius[0], 'four')
planetB = pkp.planet.keplerian(epoch(0), kep_elements[:,1], mu_star, mu_planet[1], radius[1], safe_radius[1], 'five')

time_array = np.linspace(0.1,4,1000)
v_list = []
for time_use in time_array:
    t1 = epoch(0)            #relative to t0
    t2 = epoch(time_use/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetA.eph(t1)
    r2, v2 = planetB.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)

    v1 = l.get_v1()
    vel =  np.linalg.norm(v1)/1000
    v_list.append((vel,time_use))
v_list = np.array(v_list)
print 'min v' ,np.amin(v_list, axis = 0)
i = np.argmin(v_list, axis = 0)[0]
optimal_time = v_list[i,1]
print 'optimal time use:', v_list[i,1]
fig = plt.figure()
ax = fig.gca(projection = '3d')

plot_planet(planetA, t0 = epoch(0), legend = True, units = AU, ax = ax)
plot_planet(planetB, t0 = epoch(optimal_time/DAY2YEAR), legend = True, units = AU, ax = ax)
plot_lambert(l, legend=True, units =AU, ax=ax)
plt.show()


min v [ 13.36805209   0.1       ]
optimal time use: 0.5996996997


In [88]:
t1 = epoch(0)            #relative to t0
t2 = epoch(optimal_time/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


r1, v1 = planetA.eph(t1)
r2, v2 = planetB.eph(t2)

l = pkp.lambert_problem(r1,r2,dt,mu_star)

v1 = np.array(l.get_v1())
posFunc = system.getPositionsFunction()
velFunc = system.getVelocityFunction()

planetPosA = posFunc(t)[:,4]
planetVelA = velFunc(t)[:,4]
planetPosB = posFunc(t+optimal_time)[:,5]
planetVelB = velFunc(t+optimal_time)[:,5]

first_pos   = np.load('data/pos%d.npy' %first_boost)
first_times = np.load('data/times%d.npy' %first_boost)
first_vel   = np.load('data/vel%d.npy' %first_boost)
planet4_positions = posFunc(first_times)[:,4]
rel_pos = planet4_positions - first_pos.T
rel_r = np.linalg.norm(rel_pos, axis = 0)
closest_i = np.argmin(rel_r)
pos_cl = first_pos[closest_i] 
vel_cl = first_vel[closest_i] 

print np.array(r1)
print planetPosA * AU
print v1[0,0:2], planetVelA*AU/SEC2YEAR
print vel_cl*AU/SEC2YEAR

[  3.03725975e+10   7.12047806e+11   0.00000000e+00]
[  2.86988696e+10   7.12122367e+11]
[-17738.6507897   -2729.29052777] [-17228.76052369    745.65419262]
[-13104.50809915    310.58521072]
